In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#/kaggle/input/the-great-indian-hiring-hackathon/Participants_Data_TGIH/Test.csv
#/kaggle/input/the-great-indian-hiring-hackathon/Participants_Data_TGIH/Sample Submission.csv
#/kaggle/input/the-great-indian-hiring-hackathon/Participants_Data_TGIH/Train.csv

# Data Cleaning

In [ ]:
df=pd.read_csv("/kaggle/input/the-great-indian-hiring-hackathon/Participants_Data_TGIH/Train.csv")

In [ ]:
df1=pd.read_csv("/kaggle/input/the-great-indian-hiring-hackathon/Participants_Data_TGIH/Test.csv")

In [ ]:
df1.info()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df['first_2_CID']=df['CustomerID'].apply(lambda x: int(str(x)[:2]))   
# extract the first two number of the customerid
df1['first_2_CID']=df1['CustomerID'].apply(lambda x: int(str(x)[:2])) 
#feature extrzction

In [ ]:
df.head() #there are no null values

In [ ]:
df.info()

In [ ]:
df["InvoiceDate"]=pd.to_datetime(df["InvoiceDate"])
df1["InvoiceDate"]=pd.to_datetime(df1["InvoiceDate"]) #change to date type

In [ ]:
df["InvoiceDateO"]=df["InvoiceDate"].apply(lambda x: x.toordinal())
df1["InvoiceDateO"]=df1["InvoiceDate"].apply(lambda x: x.toordinal()) #change date to ordinal data

In [ ]:
df.InvoiceNo.nunique()

In [ ]:
#df=df.sort_values("StockCode")

In [ ]:
#df[df["InvoiceDate"].dt.month==12]

In [ ]:
df.info() #all data type is now correct

In [ ]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
#df1[df1['Country']==35]

In [ ]:
for i in df[['InvoiceNo', 'StockCode', 'Description', 'Quantity',
       'UnitPrice', 'CustomerID', 'Country','InvoiceDateO']].columns:
    sns.boxplot(df[i])
    plt.show() #check for outliers 
#looks like quantity and unitprice has some outliers

In [ ]:
df[df["UnitPrice"]>5000]

In [ ]:
#df[df['InvoiceDateO']==734298]

In [ ]:
#df.drop(labels=[239556],inplace=True)

### Using this value because the data is randomly selected in for training and testing so outliers can be used for trees based algorithm you will know more which algorithm is suitable at the bottom

In [ ]:
df.drop(df[df["UnitPrice"]>10000].index,inplace=True) 
#df.reset_index(drop=True,inplace=True)

In [ ]:
#df[df["UnitPrice"]==0]

In [ ]:
#df1[df1['Quantity']<0] 
#quantity cannot be negative it could be a grabage or the quantity is by mistake negative

In [ ]:
df['Quantity']=df['Quantity'].abs()
df1['Quantity']=df1['Quantity'].abs()
#quantity has negative which is invalid and also they are not garbage value 
#converting them to positive

In [ ]:
#df.drop(df[df["Quantity"]>10000].index,inplace=True) #10k

In [ ]:
df.info()

In [ ]:
df1.describe()

In [ ]:
df1.info()

# EDA

In [ ]:
df.corr() 

linear correlation is very less w.r.t unitprice with every variable so there could be a 
non linear correlation 

In [ ]:
from scipy.stats import spearmanr, pearsonr
spearmanr(df['UnitPrice'], df['Quantity']) 

unit price and quantity has high non linear correlation after checking with all the other variables ([Spearman-RankCorrelation](http://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient))

In [ ]:
plt.figure(figsize=(17,7))
sns.scatterplot(x="Country",y="UnitPrice",data=df)  

****In country 35 ,14 and 30 there are comparatively high unit price****

In [ ]:
plt.figure(figsize=(17,7))
sns.scatterplot(x="CustomerID",y="UnitPrice",data=df) 

****Low customer id has also high unit price with low prices****

In [ ]:
plt.figure(figsize=(17,7))
sns.scatterplot(x="StockCode",y="UnitPrice",data=df)

****3700 or 3800 has also high unit price with low unitprice****

# Data Processing

In [ ]:
df1.columns

In [ ]:
X_train=df[['InvoiceDateO','Quantity','StockCode', 'Description','Country','first_2_CID']]
y_train=df1[['InvoiceDateO','Quantity','StockCode', 'Description',  'Country', 'first_2_CID']]
X_test=df['UnitPrice'].values

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
y_train = sc.fit_transform(y_train)
#X_test = sc.transform(X_test)

# ML Algorithms

 ### Compared all algorithm and got extra trees regressor with high accuracy as they use all their data in making trees                   

### You can see the other algorithm used in the appendix section

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
extra1=ExtraTreesRegressor(max_depth=200,
  n_estimators=200)
extra1.fit(X_train,X_test)
np.mean(cross_val_score(extra1,X_train,X_test,scoring = 'neg_mean_squared_error', cv= 3))#36.9 #7.5569158 #.3212 #7.55609

In [ ]:
results = extra1.predict(y_train)

In [ ]:
df1['UnitPrice']=results

In [ ]:
df1['UnitPrice'].to_csv("Submission_Final.csv",header=True,index=False)

### Used genetic algorithm for optimization for extra tree

In [ ]:
#from tpot import TPOTRegressor


#tpot_classifier = TPOTRegressor(generations= 5, population_size= 24, offspring_size= 12,
#                                 verbosity= 2, early_stop= 12,
#                                 config_dict={'sklearn.ensemble.ExtraTreesRegressor': param}, 
#                                 cv = 2)
#tpot_classifier.fit(X_train,X_test)

In [ ]:
#results=tpot_classifier.predict(y_train)

In [ ]:
#tpot_classifier.fitted_pipeline_

In [ ]:
#tpot.score(X_train, X_test)

In [ ]:
#tpot.export(path_to_pipeline)
#pickle.dump(tpot.fitted_pipeline_, open(file_name, ‘wb’))
#import pickle
#tpot.score(X_train, X_test)
#pickle.dump(tpot.fitted_pipeline_, open("tpot.pkl", "wb"))
#tpot.export("tpot.py")
#tpot.export()

# Appendix

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
classifier = KNeighborsRegressor(n_neighbors = 1, metric = 'minkowski', p = 2)
classifier.fit(X_train, X_test)
y_pred=classifier.predict(X_train)
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(X_test,y_pred))#36.9

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
model = CatBoostRegressor(
max_depth=10,
        n_estimators=100,
        learning_rate=1)
model.fit( X_train,X_test, use_best_model=True, silent=True )
y_pred=model.predict(X_train)
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(X_test,y_pred))#36.9

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
Regressor = DecisionTreeRegressor(criterion='mse')
Regressor.fit(X_train, X_test)
np.mean(cross_val_score(Regressor,X_train,X_test, scoring = 'neg_mean_squared_error', cv= 3)) #145.88

In [ ]:
y_pred=Regressor.predict(X_train)
from sklearn.metrics import mean_squared_error
mean_squared_error(X_test,y_pred)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf =  RandomForestRegressor(300)
rf.fit(X_train, X_test)
#np.mean(cross_val_score(rf,X_train,X_test,scoring = 'neg_mean_squared_error', cv= 3)) #124
y_pred=rf.predict(X_train)
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(X_test,y_pred))#36.9

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression().fit(X_train, X_test)
y_pred=rf.predict(X_train)
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(X_test,y_pred))#36.9

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
alpha = []
error = []

for i in range(1,1000):
    alpha.append(i/1)
    lml = Lasso(alpha=(i/1))
    error.append(np.mean(cross_val_score(lml,X_train,X_test, scoring = 'neg_mean_absolute_error', cv= 3)))
    
plt.plot(alpha,error)

In [ ]:
err = tuple(zip(alpha,error))
df_err = pd.DataFrame(err, columns = ['alpha','error'])
df_err[df_err.error == max(df_err.error)]

In [ ]:
lm_l = Lasso(alpha=581.0)
lm_l.fit(X_train,X_test)
y_pred=lm_l.predict(X_train)
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(X_test,y_pred))

In [ ]:
from xgboost import XGBRegressor

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
model = XGBRegressor(  
        max_depth=100,
        n_estimators=200,
        learning_rate=1.5,bootstrap=False
        )
model.fit(X_train, X_test)
#np.mean(cross_val_score(model,X_train,X_test,scoring = 'neg_mean_squared_error', cv= 3)) #103
#max_depth=6, min_child_weight=3, nthread=1,objective='reg:squarederror', subsample=0.8
y_pred=model.predict(X_train)
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(X_test,y_pred)) #6.4 #5.769 #33.5295 #57.106 #7.556915879158611

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
classifier = AdaBoostRegressor( 
        n_estimators=100,
        learning_rate=1)
classifier.fit(X_train, X_test)
#np.mean(cross_val_score(classifier,X_train,X_test,scoring = 'neg_mean_squared_error', cv= 3))
y_pred=classifier.predict(X_train)
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(X_test,y_pred))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gb_clf = GradientBoostingRegressor(
        max_depth=20,
        n_estimators=100,
        learning_rate=1)
gb_clf.fit(X_train, X_test)
#np.mean(cross_val_score(gb_clf,X_train,X_test,scoring = 'neg_mean_squared_error', cv= 3))
y_pred=gb_clf.predict(X_train)
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(X_test,y_pred))

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
#d_train = lgb.Dataset(X_train, label=X_test)
#X_train, X_test, y_train, y_test = train_test_split( X_train, X_test, test_size=0.2, random_state=42)
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l2', 'auc'],
    'learning_rate': 0.5,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100,
    "n_estimators": 100
}
gbm = lgb.LGBMRegressor(**hyper_params)
gbm.fit(X_train, X_test,
        eval_set=[(X_train, X_test)],
        eval_metric='l1',
        early_stopping_rounds=100)
np.mean(cross_val_score(gbm,X_train,X_test,scoring = 'neg_mean_squared_error', cv= 3)) 